![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [4]:
# Research notebook for random forest algorithm
# Adapted from Jansen 2020  
import warnings
warnings.filterwarnings('ignore')

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  
from talib import RSI, BBANDS, MACD, NATR, ATR, PPO, SAREXT
from scipy.stats import spearmanr, norm
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.pipeline import Pipeline
from AlgorithmImports import *
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import make_scorer
import joblib

idx = pd.IndexSlice
qb = QuantBook()
YEARS = 5
DAYS_PER_YEAR = 252

In [41]:
qb.SetStartDate(2023, 2, 22)

df = qb.History(['SPY'], 10000, resolution=Resolution.Daily) 
df['SAREXT'] = SAREXT(df.high, df.low)
df.reset_index()
df.head()



In [13]:
symbols = {}
assets = ["SHY", "TLT", "SHV", "TLH", "EDV", "BIL",
          "SPTL", "TBT", "TMF", "TMV", "TBF", "VGSH", "VGIT",
          "VGLT", "SCHO", "SCHR", "SPTS", "GOVT", 'SPY']

for i in range(len(assets)):
    symbols[assets[i]] = qb.AddEquity(assets[i],Resolution.Minute).Symbol

qb.SetStartDate(2020, 1, 1)

# qb.AddUniverse(CoarseSelectionFunction, FineSelectionFunction)
df = qb.History(qb.Securities.Keys, YEARS*DAYS_PER_YEAR, resolution=Resolution.Daily) 
df.dropna(inplace=True)

df.head()
print( 3* timedelta(365))

### Create Features

In [9]:
# calculate returns
intervals = [1, 5, 10, 21, 63]
returns = []
by_symbol = df.groupby('symbol', group_keys=False)
for t in intervals:
    returns.append(by_symbol.close.pct_change(t).to_frame(f'ret_{t}'))
returns = pd.concat(returns, axis=1)
df1 = df 
df = pd.merge(df1, returns, on=('symbol', 'time'))
returns.info()
df.head()

In [10]:
by_symbol = df.groupby('symbol', group_keys=False)
df['rsi'] = df.groupby('symbol').close.apply(RSI)
df['ppo'] = df.groupby('symbol').close.apply(PPO)
df.head()

In [11]:
# compute bollenger bands
def compute_bb(close):
    high, mid, low = BBANDS(np.log1p(close), timeperiod=20)
    return pd.DataFrame({'bb_high': high, 'bb_mid': mid, 'bb_low': low}, index=close.index)
bb_df = df.groupby('symbol').close.apply(compute_bb)
bb_df.head() 

In [12]:
def compute_atr(stock_data):
    atr = ATR(stock_data.high, stock_data.low, stock_data.close, timeperiod=14)
    return atr.sub(atr.mean()).div(atr.std())
# atr_series = by_symbol.apply(compute_atr)
df['atr'] =  by_symbol.apply(compute_atr)
df.head()

In [13]:

df['natr'] =  by_symbol.apply(lambda x: NATR(high=x.high, low=x.low, close=x.close))

In [14]:
def compute_macd(close):
    macd = MACD(close)[0]
    return macd.sub(macd.mean()).div(macd.std())

macd_series = df.groupby(level='symbol').close.apply(compute_macd)
df['macd'] = macd_series.to_frame().drop_duplicates() 

In [15]:
indicators = ['close', 'ppo', 'natr',  'rsi']
ticker = np.random.choice(df.index.get_level_values('symbol'))
(df.loc[idx[ticker, :], indicators].reset_index('symbol', drop=True)
 .plot(lw=1, subplots=True, figsize=(16, 10), title=indicators, layout=(3, 2), legend=False))
plt.suptitle(ticker, fontsize=14)
sns.despine()
plt.tight_layout()
plt.subplots_adjust(top=.95)

In [16]:
outcomes = []
by_ticker = df.groupby('symbol')
for t in intervals:
    k = f'fwd_ret_{t}'
    outcomes.append(k)
    df[k] = by_ticker[f'ret_{t}'].shift(-t)
df.head(10)

### Training Model

In [17]:
from ml_utils import *
n_splits = 30
train_period_length = 60
test_period_length = 6
lookahead = 1

cv = MultipleTimeSeriesCV(n_splits=n_splits,
                          train_period_length=train_period_length,
                          test_period_length=test_period_length,
                          lookahead=lookahead)

In [12]:
for i, (train_idx, test_idx) in enumerate(cv.split(X=df)):
    train = df.iloc[train_idx]
    train_times = train.index.get_level_values('time')
    test = df.iloc[test_idx]
    test_times = test.index.get_level_values('time')
    df1 = train.reset_index().append(test.reset_index())
    n = len(df1)
    assert n== len(df1.drop_duplicates())
    msg = f'Training: {train_times.min().date()}-{train_times.max().date()} '
    msg += f' ({train.groupby(level="symbol").size().value_counts().index[0]:,.0f} days) | '
    msg += f'Test: {test_times.min().time()}-{test_times.max().date()} '
    msg += f'({test.groupby(level="symbol").size().value_counts().index[0]:,.0f} days)'
    print(msg)
    #if i == 3:
        # break

In [22]:
rf_reg = RandomForestRegressor(n_estimators=100, 
                                max_depth=None, 
                                min_samples_split=2, 
                                min_samples_leaf=5, 
                                min_weight_fraction_leaf=0.0, 
                                max_features='auto', 
                                max_leaf_nodes=None, 
                                min_impurity_decrease=0.0, 
                                bootstrap=True, 
                                oob_score=False, 
                                n_jobs=-1, 
                                random_state=None, 
                                verbose=0, 
                                warm_start=False)
df.head()

In [26]:
df.dropna(inplace=True)
target = sorted(df.filter(like='fwd').columns)
features = df.columns.difference(target).tolist()

X = df.drop(target, axis=1)
y = df.fwd_ret_1.to_frame()
X.head()
y.head()  

rf_reg.fit(X, y)


In [38]:
returns_map = {}
for index, row in X.iterrows():
    symbol = index[0].split()[0]
    row_df = row.to_frame().transpose() 
    cur_returns = rf_reg.predict(row_df) 
    returns_map[symbol] = cur_returns

print(returns_map)

In [ ]:

def rank_correl(y, y_pred):
    return spearmanr(y, y_pred)[0]

ic = make_scorer(rank_correl)
cv_score = cross_val_score(estimator=rf_reg,
                           X=X,
                           y=y,
                           scoring=ic,
                           cv=cv,
                           n_jobs=-1,
                           verbose=1)


In [ ]:
print(np.mean(cv_score))


In [ ]:
param_grid = {'n_estimators': [50, 100, 250],
              'max_depth': [5, 15, None],
              'min_samples_leaf': [5, 25, 100]}
gridsearch_reg = GridSearchCV(estimator=rf_reg,
                      param_grid=param_grid,
                      scoring=ic,
                      n_jobs=-1,
                      cv=cv,
                      refit=True,
                      return_train_score=True,
                      verbose=1)

In [ ]:
gs_reg = gridsearch_reg
gridsearch_reg.fit(X=X, y=y)

In [ ]:
gridsearch_reg.best_params_